In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import letter_recognition.nn.activation as activation
import letter_recognition.nn.layers as nn
import letter_recognition.nn.loss as loss
from letter_recognition import RNG

%matplotlib inline

In [ ]:
# Load data
with open("letter_recognition/data/numpy/data.npy", "rb") as f:
    images = np.load(f)
with open("letter_recognition/data/numpy/labels.npy", "rb") as f:
    labels = np.load(f)
print(len(images), len(labels))

In [ ]:
train_percentage = 0.67
split_point = int(len(images) * 0.67)
x_train, y_train = images[:split_point], labels[:split_point]
x_test, y_test = images[split_point:], labels[split_point:]
print(len(x_train), len(y_train))

In [ ]:
# Model definition
conv1_out_channels = 8
conv1_kernel_size = (3, 3)
max_pool1_size = 2
linear1_in_feat = conv1_out_channels * 13 * 13
linear1_out_feat = 128
lr = 0.01
batch_size = 8
epoch_number = 4

# Model architecture
conv1 = nn.Conv2d(1, conv1_out_channels, conv1_kernel_size)
relu = activation.ReLU()
maxpool1 = nn.MaxPool2d(max_pool1_size)
linear1 = nn.Linear(linear1_in_feat, linear1_out_feat)
linear2 = nn.Linear(linear1_out_feat, 26)
criterion = loss.CrossEntropy()

In [ ]:
scale_factor = 128
x_train = x_train[:len(x_train) // scale_factor]
y_train = y_train[:len(y_train) // scale_factor]

fig, ax = plt.subplots()
epoch_list = []
train_loss_list = []
loss_sum = 0

for epoch in tqdm(range(epoch_number)):
    # Shuffle
    p = RNG.permutation(len(x_train))
    x_train = x_train[p]
    y_train = y_train[p]

    # Batch split
    x_batches = np.array_split(x_train, len(x_train) // batch_size)
    y_batches = np.array_split(y_train, len(y_train) // batch_size)

    # Train
    for x, y in zip(x_batches, y_batches):        
        # Forward pass
        out_conv1 = conv1.forward(x)
        out_relu1 = relu.forward(out_conv1)
        out_maxpool1, idx_maxpool1 = maxpool1.forward(out_relu1)
        out_maxpool1_reshaped = out_maxpool1.reshape(out_maxpool1.shape[0], linear1_in_feat)
        out_linear1 = linear1.forward(out_maxpool1_reshaped)
        out_relu2 = relu.forward(out_linear1)
        out_linear2 = linear2.forward(out_relu2)
        current_loss = criterion.calculate(out_linear2, y)

        # Gradient calculation
        dx_loss = criterion.backward(out_linear2, y)
        dx_linear2, dw_linear2, db_linear2 = linear2.backward(dx_loss, out_relu2)
        dx_relu2 = relu.backward(dx_linear2, out_linear1)
        dx_linear1, dw_linear1, db_linear1 = linear1.backward(dx_relu2, out_maxpool1_reshaped)
        dx_linear1 = dx_linear1.reshape(out_maxpool1.shape)
        dx_maxpool1 = maxpool1.backward(dx_linear1, out_relu1, idx_maxpool1)
        dx_relu1 = relu.backward(dx_maxpool1, out_conv1)
        _, dw_conv1, db_conv1 = conv1.backward(dx_relu1, x)

        # SGD
        conv1.weight -= lr * dw_conv1
        conv1.bias -= lr * db_conv1
        linear1.weight -= lr * dw_linear1
        linear1.bias -= lr * db_linear1
        linear2.weight -= lr * dw_linear2
        linear2.bias -= lr * db_linear2

        # Loss tracking
        loss_sum += current_loss

    epoch_list.append(epoch + 1)
    train_loss_list.append(loss_sum / len(x_batches))
    loss_sum = 0

ax.plot(epoch_list, train_loss_list)
ax.set(xlabel="Epoch", ylabel="Loss")
fig.savefig("figs/model1.jpg")
plt.show()